In [ ]:
!pip install geopandas

In [2]:
import pandas as pd
import pandas as pd
import geopandas as gpd 
import numpy as np 
import matplotlib.pyplot as plt

In [ ]:
!pip install --upgrade botocore==1.23.26  #Sur colab, sinon bug
!pip install --upgrade urllib3==1.22.0 #Sur colab, sinon bug
!pip install py7zr #Sur colab, sinon bug
!pip install s3fs #Sur colab, sinon bug
!git clone https://github.com/InseeFrLab/cartogether.git
%cd ./cartogether
!pip install -r requirements.txt
!pip install .

In [4]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [45]:
#Base adresses -> coordonnées GPS
df_adresse=pd.read_csv('/content/adresses-13.csv', sep =";") 

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [46]:
df_adresse.head()

,id,id_fantoir,numero,rep,nom_voie,code_postal,code_insee,nom_commune,code_insee_ancienne_commune,nom_ancienne_commune,...,lat,type_position,alias,nom_ld,libelle_acheminement,nom_afnor,source_position,source_nom_voie,certification_commune,cad_parcelles
0,13003_snil1g_00003,NaN,3,NaN,Quartier sous Ville,13980,13003,Alleins,NaN,NaN,...,43.706169,segment,NaN,NaN,ALLEINS,QUARTIER SOUS VILLE,inconnue,inconnue,0,NaN
1,13003_snil1g_00012,NaN,12,NaN,Quartier sous Ville,13980,13003,Alleins,NaN,NaN,...,43.706076,segment,NaN,NaN,ALLEINS,QUARTIER SOUS VILLE,inconnue,inconnue,0,NaN
2,13003_snil1g_00014,NaN,14,NaN,Quartier sous Ville,13980,13003,Alleins,NaN,NaN,...,43.706076,segment,NaN,NaN,ALLEINS,QUARTIER SOUS VILLE,inconnue,inconnue,0,NaN
3,13003_snil1g_00270,NaN,270,NaN,Quartier sous Ville,13980,13003,Alleins,NaN,NaN,...,43.706632,segment,NaN,NaN,ALLEINS,QUARTIER SOUS VILLE,inconnue,inconnue,0,NaN
4,13003_snil1g_00271,NaN,271,NaN,Quartier sous Ville,13980,13003,Alleins,NaN,NaN,...,43.706645,segment,NaN,NaN,ALLEINS,QUARTIER SOUS VILLE,inconnue,inconnue,0,NaN


In [36]:
#Base risque d'inondations niveau national
df_risque_inond=gpd.read_file("/content/n_carte_inond_s.shp") 

In [39]:
df_risque_inond["scenario"].drop_duplicates()

0            None
1           01For
3           02Moy
6           04Fai
10     04Faicc_ct
11       03Mcc_ct
50          03Mcc
138    01Forcc_ct
Name: scenario, dtype: object

In [40]:
def adresse_to_geometry_bis(dep,num,nom_voie,ville,code_postal):
  lon =df_adresse[(df_adresse["numero"] == num) & (df_adresse["nom_commune"] == ville) & (df_adresse["nom_voie"] == nom_voie)]["lon"]
  lat=df_adresse[(df_adresse["numero"] == num) & (df_adresse["nom_commune"] == ville) & (df_adresse["nom_voie"] == nom_voie)]["lat"]
  return(Point(lon,lat))

In [41]:
#Fonction geometry ->risque 
def risque(geometry,df_risque_inond):
  risque=[]
  for k in range(1,len(df_risque_inond["geometry"])):
    if df_risque_inond["geometry"][k].contains(geometry):
      risque.append(df_risque_inond["scenario"][k])
  
  if len(risque)==0:
    return(3)
  
  elif risque[0]=="04Faicc_ct" or risque[0]=="04Fai":
    return(6)

  elif risque[0]=="03Mcc_ct" or risque[0]=="03Mcc":
    return(8)
  
  elif risque[0]=="02Moy":
    return(12)
  
  else : 
    return(18)
  


In [42]:
#Fonction finale
def adresse_to_risque(dep,num,nom_voie,ville,code_postal,df_risque_inond):
  return(risque(adresse_to_geometry_bis(dep,num,nom_voie,ville,code_postal),df_risque_inond))



In [47]:
adresse_to_risque("13",3,"Quartier sous Ville","Alleins",13980,df_risque_inond)

3